<h1> ENRON EMAILS: INVERTED INDEX AND WEIGHT OF EVIDENCE <h1>

<h2> Introduction <h2>

The main objective of this notebook is to carry out an Exploratory Data Analysis (EDA) on Enron email corpus. This corpus is a subset from emails dataset  acquired during the investigation after the company financial collapse in 2001. In this subset, every document is the body of an email and it is labelled as 1 if it is related to energy topic (and therefore relevant in the trial) or 0 if it is not. The data is taken from the course in EDx Analytics Edge:

https://www.edx.org/es/course/analytics-edge-mitx-15-071x-3

Our task is to build a model that allow us to predict whether a email is relevant to the trial or not, so a subset of documents have been labelled in order to train the model and then the model can be applied to the rest of the data. In order to go at the model building phase, we will explore our subset of data to achiveve two main goals:
1. Gain some business knowledge about our corpus, for example, we will discover important events in Enron history and link them to our exploration.
2. Obtain an analytical insight that will allow us to take sound decisions during the model building step, intead of just hardcoding some recommended parameters values.

https://en.wikipedia.org/wiki/Enron_Corpus

To do that, are going to develop the following analytical tools:
* Build an Inverted Index:  This will allow us to easily and efficiently explore the relationship between words (tokens) and emails (documents).
* Compute Weigth Of Evidence: To determine each token's predictive power.
* Use Brunel visualizations to extract useful insights that we can leverage in a model building phase.

One of our objectives is to discover what happened to Enron, so I will not enter in detail, but here is a quick context that we are going to link to our exploration:

Enron Corporation was an energy company from the USA, that, in late 90's experimented an exponential growth, triggered by an energy deregulation process in the USA. Accounting among it's main activities there were natural gas and power trading, power generation and distribution, and developing a wide variety of other innovative and risky business ideas. Suddenly, in 2001, Enron goes  bankrupt, accused of accounting fraud, power market manipulation, and triggering an energy supply crisis.

https://en.wikipedia.org/wiki/Enron#California.27s_deregulation_and_subsequent_energy_crisis


<h2> 1. Load data and visualization tools <h2>

Obviously, the first stept is to donwload the data from the github repository as a textFile. This file is a pipe "|" separated file with three fields: id, email, label.

Then we are going to add a jar in order to use Brunel visualizations. It is an easy way of ploting Spark DataFrames in a Scala notebook, just as with pixiedust in PySpark notebooks. Brunel has its own interface and language, but using the Brunel Visualization Cookbook, it will be very easy to learn the basics: https://github.com/Brunel-Visualization/Brunel/wiki/Brunel-Visualization-Cookbook

* Brunel: https://github.com/Brunel-Visualization/Brunel
* Pixiedust: https://github.com/ibm-watson-data-lab/pixiedust


In [1]:
//val spark = new org.apache.spark.sql.SQLContext(sc)
//import spark.implicits._

import org.apache.spark.sql._
import org.apache.spark.sql.functions._
import org.apache.spark.sql.types._
import org.apache.spark.sql.expressions.Window
import org.apache.spark.ml.feature.{StopWordsRemover, Bucketizer}
import org.apache.spark.ml.Pipeline

In [2]:
//I recommend using Brunel 2.2 with Spark 2.1.
%AddJar -magic https://brunelvis.org/jar/spark-kernel-brunel-all-2.2.jar

Using cached version of spark-kernel-brunel-all-2.2.jar


In [3]:
//Download data from github to the local filesystem of the notebook, root folder
import sys.process._
import java.net.URL
import java.io.File

def fileDownloader(url: String, filename: String) = {
    new URL(url) #> new File(filename) !!
}

fileDownloader("https://raw.githubusercontent.com/manualrg/DSLAB_TextAnalytics/master/enron_textfile.txt", "enron_textfile.txt")

""

The first part of this notebook will be carried out in RDD API, as we want to analyze unstructured data and transform it to an structured dataset (our inverted index). So we are going to load the data with textFile as a RDD[String], split every record by "|", and them map it to tuples of three elements: (id[Int], doc[String], label[Double]).

In [4]:
val corpus = sc.textFile("enron_textfile.txt").
    map(_.split("\\|")).
    map(arr => (arr(0).toInt, arr(1), arr(2).toDouble) )
corpus.persist()
corpus.take(1)

Waiting for a Spark session to start...

Array((1,North America's integrated electricity market requires cooperation on environmental policies Commission for Environmental Cooperation releases working paper on North America's electricity market Montreal, 27 November 2001 -- The North American Commission for Environmental Cooperation (CEC) is releasing a working paper highlighting the trend towards increasing trade, competition and cross-border investment in electricity between Canada, Mexico and the United States. It is hoped that the working paper, Environmental Challenges and Opportunities in the Evolving North American Electricity Market, will stimulate public discussion around a CEC symposium of the same title about the need to coordinate environmental policies trinationally as a North...

<h2> 2. Building an Inverted Index<h2>

An inverted index is a data structure that maps every token to the collection of documents (represented as an Id) were that token appears. The formal definition is here: https://en.wikipedia.org/wiki/Inverted_index.

Let's see an example:

Our corpus:
1. Scala is functional, but functional programming is difficult
2. Spark is written in Scala
3. Python is cooler than R

Firstly, every document is splitted into a collection of tokens, this step is called Tokenization:
1. Array("Scala" "is" "functional", "but", "functional", "programming", "is", "difficult")
2. Array("Spark", "is", "written", "in", "Scala")
3. Array("Python", "is", "cooler", "than", "R")

Then every word in the corpus is mapped to the collection of document-ids where it appears
1. "Scala" -> (1, 2)
2. "is" -> (1, 2, 3)
3. "functional" -> (1)
4. "Spark" -> (2)
...

Consider that, in a large corpus, the number of documents may be higher than the number of words in a language, so inverted indexes are  very popular techniques in large scale search engine and text analytics in general. Then it will be easy to build a frequency ranking and compute some useful metrics:
* Tokenize and clean documents
* Build an inverted index: Map every token to the collection of documents (represented by a tuple (id, label))
* Build a frequency ranking: For each token, take the set of documents and compute frequency, then sort the ranking in descending order.

The first step is tokenization, the following data preprocessing operations will be considered:
* remove non alphanumeric characters
* remome multiple blank spaces
* remove trailing and leading blanks
* transform every character to lower case
* split by word, so every token is considered to be between blank spaces
* filter tokens contained in stopwords set

In [5]:
//Get a stopwords list from the feature transformer StopWordsRemover from ml library
val remover = new StopWordsRemover()
val stopw = remover.getStopWords
println("Number of stopwords: " + stopw.size)
//Display ten first stopwords, this a set of pronoums, prepositions, etc. the kind of word that may appear in every document and do not add any predictive power
stopw.slice(0,10)

Number of stopwords: 153


Array(i, me, my, myself, we, our, ours, ourselves, you, your)

To take advange of distributed data processing, we will convert our stopwords collection in a broadcasted variable, so it will be copied to every worker node and reduce data transference over the network

In [6]:
import org.apache.spark.broadcast.Broadcast
val stop_BC :Broadcast[Set[String]]= sc.broadcast(stopw.toSet)

In [7]:
//Perform basic data cleaning tasks in every document
//Split every document into tokens: Array[String]
//filter form every array a set of stopwords

def tokenize(doc :String, stopwords: Broadcast[Set[String]]) : Array[String] = {
  doc.replaceAll("[^a-zA-Z0-9]", " ").
    replaceAll("//s{2,}", " "). 
    trim().
    toLowerCase().
    split("\\W+").
    filterNot(stopwords.value.contains(_)) 
    //id[Int], doc[String], tokens[Array[String]], label[Double]
}

In [8]:
//dummy example
val excorpus = sc.parallelize(Seq((0, "Scala is functional, but functional programming is difficult", 1.0), 
                                (1, "Spark is written in Scala", 1.0), 
                                (2, "@Python is cooler than R", 0.0)) )

val exrdd_tokens =excorpus.map(doc => (doc._1, doc._2 ,tokenize(doc._2, stop_BC), doc._3) )//label, doc, tokens
exrdd_tokens.collect()

Array((0,Scala is functional, but functional programming is difficult,Array(scala, functional, functional, programming, difficult),1.0), (1,Spark is written in Scala,Array(spark, written, scala),1.0), (2,@Python is cooler than R,Array(python, cooler, r),0.0))

The second step, is to build the inverted Index:

In [9]:
//Build an inverted index: Maps every token to a collection of every pairs of (document-id, label) where it appears
def invIdx(rdd :org.apache.spark.rdd.RDD[(Int, String, Array[String], Double)]) :org.apache.spark.rdd.RDD[(String,Array[(Int, Double)])] = {
    //original strucutre: RDD[ (id[Int], Doc[String], tokens[Array], label[Double]) ]
    //output structure: RDD[token[String] -> collection[(doc-id[Int], label[Double])]]
    rdd.map(doc => (doc._3.map(tok => (tok, (doc._1, doc._4) ) 
                            ) 
                 )
         ).flatMap(x => x).groupByKey().map(idx => (idx._1, idx._2.toArray))
}



In [10]:
//dummy example: Build inverted index
val exinvIdx1_rdd = invIdx(exrdd_tokens)
println("Inverted Index: token -> set[Document]")
println("Document: (id[Int], label[Double])")
exinvIdx1_rdd.collect().map(x => (x._1, x._2.mkString(", ")))foreach(println)

Inverted Index: token -> set[Document]
Document: (id[Int], label[Double])
(scala,(0,1.0), (1,1.0))
(cooler,(2,0.0))
(python,(2,0.0))
(difficult,(0,1.0))
(r,(2,0.0))
(spark,(1,1.0))
(written,(1,1.0))
(programming,(0,1.0))
(functional,(0,1.0), (0,1.0))


Finally, is time to construct a frequency ranking. To do that we will leverage the set of documents that every token is mapped to. Notice that a token may appear several times in the same document (like functional in our dummy example), so let's compute the following metrics:

* freq: Total number of times that a token j appears in corpus C
* ndocs: Number of documents where token j appears.
* n1: number of label=1 documents where token j appears
* n0: number of label=0 documents where token j appears
* doc-freq: Number of times that token j appears in document i

Obviously: ndocs = n1 + n0

Finally the structure of our ranking will be:

<center> ( token[String], freq, ndocs, n1, n0, Set[(Doc-id[Int], label[Double], doc-freq[Int]] ) </center>

In [11]:
def docFreq(docIt: Array[(Int, Double)]) : Set[(Int, Double, Int)] = {
    docIt.groupBy(identity).
        map(pair => (pair._1 -> pair._2.size)).
        map(x => (x._1._1, x._1._2, x._2) ).toSet
}


In [12]:
//dummy example: Build inverted index with docFreq
val exinvIdx2_rdd = exinvIdx1_rdd.map(pair => (pair._1, docFreq(pair._2)))
println("Inverted Index with docFreq: token -> set[Document]")
println("Document: (id[Int], label[Double], docFreq[Int])")
exinvIdx2_rdd.collect().foreach(println)

Inverted Index with docFreq: token -> set[Document]
Document: (id[Int], label[Double], docFreq[Int])
(scala,Set((1,1.0,1), (0,1.0,1)))
(cooler,Set((2,0.0,1)))
(python,Set((2,0.0,1)))
(difficult,Set((0,1.0,1)))
(r,Set((2,0.0,1)))
(spark,Set((1,1.0,1)))
(written,Set((1,1.0,1)))
(programming,Set((0,1.0,1)))
(functional,Set((0,1.0,2)))


In [13]:
//dummy example: Frequency ranking
val exrank = exinvIdx2_rdd.map(idx =>{val ndocs=  idx._2.size
                                            val docs = idx._2.toArray
                                            val freq = docs.map(doc =>  doc._3).sum
                                            val n1 = docs.toArray.map(it => it._2).sum
                                            val n0 = docs.filter(it => it._2 == 0).size.toDouble
                                            val doc_ids = docs.map(it => it._1)
                                            val labels = docs.map(it => it._2)
                                            (idx._1, ndocs, freq, n1, n0, idx._2)
                              } ).sortBy(-_._3)
println("Frequency Ranking: (token[String], freq[Int]), ndocs[Int], n1[Double], n0[Double], Set[Document]")
println("Document: (id[Int], label[Double], docFreq[Int])")
exrank.collect().foreach(println)

Frequency Ranking: (token[String], freq[Int]), ndocs[Int], n1[Double], n0[Double], Set[Document]
Document: (id[Int], label[Double], docFreq[Int])
(scala,2,2,2.0,0.0,Set((1,1.0,1), (0,1.0,1)))
(functional,1,2,1.0,0.0,Set((0,1.0,2)))
(cooler,1,1,0.0,1.0,Set((2,0.0,1)))
(python,1,1,0.0,1.0,Set((2,0.0,1)))
(difficult,1,1,1.0,0.0,Set((0,1.0,1)))
(r,1,1,0.0,1.0,Set((2,0.0,1)))
(spark,1,1,1.0,0.0,Set((1,1.0,1)))
(written,1,1,1.0,0.0,Set((1,1.0,1)))
(programming,1,1,1.0,0.0,Set((0,1.0,1)))


Perform the same calculations to Enron corpus:

<h3> 1.Tokenization <h3>

In [14]:
val rdd_tokens = corpus.map(doc => (doc._1, doc._2 ,tokenize(doc._2, stop_BC), doc._3) )//label, doc, tokens
val record= rdd_tokens.take(1)(0)
println("Document id: "+ record._1)
println("Document label: "+ record._4)
println("First 50 characters from document: "+ record._2.slice(0,50))
println("First 10 tokens: " + record._3.slice(0,10).mkString(", "))

Document id: 1
Document label: 0.0
First 50 characters from document: North America's integrated electricity market requ
First 10 tokens: north, america, integrated, electricity, market, requires, cooperation, environmental, policies, commission


<h3> 2.Build an inverted index, mapping to sets of tuples consisting in: doc-id, doc-label, and doc-freq <h3>

In [15]:
val invIdx_rdd = invIdx(rdd_tokens).map(pair => (pair._1, docFreq(pair._2) )
                                       )
val ntokens = invIdx_rdd.count()
println("Number of tokens in the corpus: " + ntokens)
println("Display the first token from the inverted index")
invIdx_rdd.take(1).foreach(println)

[Stage 13:>                                                         (0 + 2) / 2]Number of tokens in the corpus: 21575
Display the first token from the inverted index
(weirton,Set((67,0.0,1)))


<h3> 3.Compute a Frequency Ranking <h3>

In [16]:
val rank_metrics_rdd = invIdx_rdd.map(idx =>{val ndocs=  idx._2.size
                                            val docs = idx._2.toArray
                                            val freq = docs.map(doc =>  doc._3).sum
                                            val n1 = docs.toArray.map(it => it._2).sum
                                            val n0 = docs.filter(it => it._2 == 0).size.toDouble
                                            val doc_ids = docs.map(it => it._1)
                                            val labels = docs.map(it => it._2)
                                            (idx._1, ndocs, freq, n1, n0, idx._2)
                              } ).sortBy(-_._3)
//(token, ndocs, n1, n0, set-of-documents)
rank_metrics_rdd.take(1)

[Stage 19:=============================>                            (1 + 1) / 2]

Array((enron,512,4977,95.0,417.0,Set((225,0.0,22), (307,0.0,14), (794,0.0,10), (266,0.0,1), (753,0.0,8), (855,0.0,2), (735,0.0,2), (34,0.0,14), (294,0.0,8), (383,0.0,4), (816,0.0,26), (646,0.0,12), (235,0.0,90), (551,1.0,17), (281,0.0,7), (276,1.0,2), (490,1.0,4), (80,0.0,4), (514,0.0,18), (616,0.0,4), (306,0.0,2), (672,0.0,1), (282,0.0,1), (460,0.0,2), (83,0.0,8), (190,0.0,41), (732,0.0,3), (204,1.0,10), (210,0.0,68), (253,0.0,3), (472,0.0,1), (430,0.0,3), (113,0.0,12), (96,0.0,8), (650,0.0,3), (778,0.0,6), (700,0.0,9), (598,0.0,7), (812,0.0,4), (108,0.0,2), (576,1.0,5), (516,0.0,11), (23,1.0,4), (574,0.0,4), (458,1.0,1), (256,0.0,1), (123,0.0,2), (351,0.0,28), (491,0.0,9), (511,0.0,8), (556,0.0,12), (564,0.0,1...

Good Job! As tuples can be cumbersome , let's use DataFrame API and obtain the following metrics:
* e: Number of label=1 documents (or events) where token-j appears (n1)
* ne: Number of label=0 documents (or non-events) where token-j appears (n0)
* eprop: Proportion of label=1 documents in token-j document set
* logFreq: Frequency in log10 scale
* freq_bins: order of magnitude of frequency
* ndocs: Number of documents where token-j appears
* docFreq_bins: order of magnitude of ndocs
* logfreq_bins: Integer part of logFreq
* rank: Position in frequency ranking

In [17]:
//Structure of rank_metrics_rdd: RDD[(token[String], frequency[Long], n1[Long], n0[Long], collection[(id[Int], label[Double])])]
//Create a frequency rank with windowing functions
val wSpec =  Window.orderBy(col("frequency").desc)
val rank_df = spark.createDataFrame(rank_metrics_rdd.map(idx => (idx._1, idx._2, idx._3, idx._4, idx._5))).toDF("token","ndocs","frequency","e","ne").
    withColumn("eprop", $"e"/($"e"+$"ne")).
    withColumn("logFreq", log10($"frequency")).
    withColumn("freq_bins", floor(log10($"frequency")).cast(StringType) ).
    withColumn("ndocs_bins", floor(log10($"ndocs")).cast(StringType) ).
    withColumn("logfreq_bins", floor($"logFreq").cast(StringType) ).
    withColumn("rank", row_number().over(wSpec))
   
rank_df.persist()

println("")
println("Top 10  most frequent tokens:")
rank_df.show(10)

Top 10  most frequent tokens:
[Stage 23:=============================>                            (1 + 1) / 2]+-------+-----+---------+-----+-----+-------------------+------------------+---------+----------+------------+----+
|  token|ndocs|frequency|    e|   ne|              eprop|           logFreq|freq_bins|ndocs_bins|logfreq_bins|rank|
+-------+-----+---------+-----+-----+-------------------+------------------+---------+----------+------------+----+
|  enron|  512|     4977| 95.0|417.0|        0.185546875| 3.696967640744023|        3|         2|           3|   1|
|    ect|  361|     4131| 52.0|309.0| 0.1440443213296399|3.6160551949765862|        3|         2|           3|   2|
|    com|  332|     3625| 80.0|252.0|0.24096385542168675|3.5593080109070123|        3|         2|           3|   3|
|    hou|  329|     2051| 46.0|283.0| 0.1398176291793313|3.3119656603683665|        3|         2|           3|   4|
|      e|  268|     1318| 66.0|202.0| 0.2462686567164179| 3.119915410257991|  

To conclude this step, let's perform some visualizations with Brunel in order to answer these questions:
1. How does the token frequency distribution look like?
2. Is there a relationship between token frequency and the propotion of events (proportion of documents where token-j appears labelled as 1)?

<h3> 1.Univariate Frequency Analysis <h3>

Let's leverage Brunel visualizations utilities, such as managing the amount of data to be displayed. This is a feature of the utmost importance in a Big Data framework, when our DataFrames may contain millions of rows.

In [18]:
%%brunel
data("rank_df") bar x(token) y(frequency) sort(frequency) bottom(rank:10) 
title("Frequency ranking of top 10 tokens") axes(x:'token',y:'Freq') interaction(none)

In [19]:
%%brunel
data("rank_df") bar x(token) y(ndocs) sort(ndocs) bottom(rank:10) 
title("Ndocs ranking of top 10 tokens") axes(x:'token',y:'Freq') interaction(none)

In [20]:
%%brunel
data("rank_df") bar x(freq_bins) y(#count) sort(freq_bins) color(ndocs_bins)
title("Relationship between frequency and docs") axes(x:'Frequency order of magnitude',y:'Count')

As we expected, high frequency terms appear also in the higher number of documents, but rankings may not be de same (enron is the first in frequency ranking with almost 5000 times, but second in ndoc ranking, appearing in over 500 documents. Ndocs are smoothly distributed over frequency.

In [21]:
%%brunel
data("rank_df") area x(rank) y(frequency) sort(frequency) bottom(rank:10000)
title("Frequency ranking of top 10.000 tokens") axes(x:'rank-position',y:'Freq') interaction(none)

In y-axis, it is displayed absolute frequency and in x-axis token positon in frecuency rank, only top 10.000 most frequent tokens are displayed. We can see that there are few very frequent terms and then frequency decreases exponentially (positive skewness). In fact, there are only 7 tokens that appear more than 1,000 times.

This is a very important insight, because in modelling phase, we will use bag of words (BOW) featurization, it consists in computing each token-j frequency in document-i and building a document-term matrix. The set of tokens is called vocabulary and it is the number of columns in that matrix. There are three approaches:
1. Using full vocabulary set
2. Developing a custom vocabulary set
3. Using a subset of the vocabulary, based in token frequency (for example top 500 most frequent tokens in the corpus)

If we use the third approach, this frequency study is foundamental.

We can analyze the full vocabulary frequency plotting a frequency distribution or plotting the y-axis in log scale to smooth the visualization:

In [22]:
%%brunel
data("rank_df") bar x(frequency) y(#count) bin(frequency: 50) sort(frequency) 
title("Frequency distribution") axes(x:'frequency',y:'count') 

In [23]:
%%brunel
data("rank_df") area x(rank) y(logFreq) sort(logFreq)
title("Log Scale Frequency ") axes(x:'rank-position',y:'Freq') interaction(none)

In [24]:
//Remember that a log10(x)=0 when x=1, that is the meaning of the long  zero-value tail from 12,000 to 22,000
rank_df.groupBy($"logFreq_bins").agg(min("frequency"), max("frequency"), count("*")).show()

+------------+--------------+--------------+--------+
|logFreq_bins|min(frequency)|max(frequency)|count(1)|
+------------+--------------+--------------+--------+
|           3|          1058|          4977|       7|
|           2|           100|           992|     363|
|           1|            10|            98|    3130|
|           0|             1|             9|   18075|
+------------+--------------+--------------+--------+



<h3> 2. Relationship between token frequency and the propotion of events<h3>

We have performed a univariate analysis of token frequency, now it is time to study the relationship between token frequency and the proportion of label=1 documents that token-j appears.

In [25]:
%%brunel
data("rank_df") point x(eprop) y(frequency) sort(eprop) 
title("Scatterplot: freq-event") axes(x:'event proportion',y:'frequency') interaction(none)

It can be seen that there is no relationship between token frequency and predictive power, however:
* The top 5 frequency terms seem to be in label=0 emails
* There are a set of tokens that only appear in label=1 or label=0 documents, and their frequency is less than 100.

Let's check those  insights out:

In [26]:
println("Top 10 most frequent tokens:")
rank_df.filter($"rank"<=lit(10)).show()

Top 10 most frequent tokens:
+-------+-----+---------+-----+-----+-------------------+------------------+---------+----------+------------+----+
|  token|ndocs|frequency|    e|   ne|              eprop|           logFreq|freq_bins|ndocs_bins|logfreq_bins|rank|
+-------+-----+---------+-----+-----+-------------------+------------------+---------+----------+------------+----+
|  enron|  512|     4977| 95.0|417.0|        0.185546875| 3.696967640744023|        3|         2|           3|   1|
|    ect|  361|     4131| 52.0|309.0| 0.1440443213296399|3.6160551949765862|        3|         2|           3|   2|
|    com|  332|     3625| 80.0|252.0|0.24096385542168675|3.5593080109070123|        3|         2|           3|   3|
|    hou|  329|     2051| 46.0|283.0| 0.1398176291793313|3.3119656603683665|        3|         2|           3|   4|
|      e|  268|     1318| 66.0|202.0| 0.2462686567164179| 3.119915410257991|        3|         2|           3|   5|
|  power|  196|     1213| 76.0|120.0| 0.387

In fact, from the top 10 most frequent tokens, only power and energy appear mostly in label=1 documents.

In [27]:
println("Tokens that appear only in label=0 documents:")
rank_df.filter($"eprop"===lit(0)).orderBy($"frequency".desc).show(10)

Tokens that appear only in label=0 documents:
+-------------+-----+---------+---+----+-----+------------------+---------+----------+------------+----+
|        token|ndocs|frequency|  e|  ne|eprop|           logFreq|freq_bins|ndocs_bins|logfreq_bins|rank|
+-------------+-----+---------+---+----+-----+------------------+---------+----------+------------+----+
|         sara|   39|      127|0.0|39.0|  0.0| 2.103803720955957|        2|         1|           2| 266|
|         tana|   39|      101|0.0|39.0|  0.0|2.0043213737826426|        2|         1|           2| 362|
|          wpd|   11|      100|0.0|11.0|  0.0|               2.0|        2|         1|           2| 369|
|   shackleton|   31|       97|0.0|31.0|  0.0|1.9867717342662448|        1|         1|           1| 373|
|       folder|    2|       93|0.0| 2.0|  0.0|1.9684829485539352|        1|         0|           1| 392|
|synchronizing|    1|       92|0.0| 1.0|  0.0|1.9637878273455551|        1|         0|           1| 403|
|        

In the list of tokens that appear only in label=0 documents there are some names and, in general, terms not related to energy or electricity, so those documents may not be useful in the trial

In [28]:
println("Tokens that appear only in label=1 documents:")
rank_df.filter($"eprop"===lit(1)).orderBy($"frequency".desc).show(10)

Tokens that appear only in label=1 documents:
+------------+-----+---------+----+---+-----+------------------+---------+----------+------------+----+
|       token|ndocs|frequency|   e| ne|eprop|           logFreq|freq_bins|ndocs_bins|logfreq_bins|rank|
+------------+-----+---------+----+---+-----+------------------+---------+----------+------------+----+
|     numerix|    1|       41| 1.0|0.0|  1.0|1.6127838567197355|        1|         0|           1| 967|
|      schenk|    1|       33| 1.0|0.0|  1.0|1.5185139398778875|        1|         0|           1|1203|
|       solar|    2|       29| 2.0|0.0|  1.0| 1.462397997898956|        1|         0|           1|1369|
|     pergher|    6|       27| 6.0|0.0|  1.0|1.4313637641589874|        1|         0|           1|1455|
|      unfair|    4|       27| 4.0|0.0|  1.0|1.4313637641589874|        1|         0|           1|1468|
|     reuters|    5|       26| 5.0|0.0|  1.0| 1.414973347970818|        1|         0|           1|1485|
|californians|    

However, in this case, there are some interesting terms, such us californians (rolling blackouts were spread accross California in order to spike electricity price and take advantage of arbitrage opportunities in power generation market) and reuters (related to their energy trading unfair strategies). 

Well done! There is only one more step ahead, to computate Weigth of Evidence

<h2> 3. Computing WOE: Weigth Of Evidence <h2>

Weight of Evidence (WOE) is a predictive power metric that considers each predictor independent contribution to the (binary) response variable (here known as label). Originally, it is applied to categorical predictors, and computations are performed to every level of the predictor. In this scenario, we will compute WOE, considering that the token column  is a categorical predictor, and each term is a level. It can be calculated as follows:

* Given a predictor X<sub>j</sub>, with levels l<sub>i</sub> i=1,...,L,
* n<sub>1</sub> is the number of events i a given level, and n<sub>0</sub> is the number of non-events
* N<sub>1</sub> and N<sub>0</sub> are total number of events in the dataset

$$ WOE_{ij} = log(\frac{P(X_{j} = l_{i} | y=1)}{P(X_{j} = l_{i} | y=0)})$$
Regarding feature X<sub>j</sub>:
$$P(X_{j} = l_{i}| y=1) \text{ is the conditional probabilty of } level_{i} \text{ given label=1: } P_{1}=\frac{n_{1}}{N_{1}}$$

$$P(X_{j} = l_{i}| y=0) \text{ is the conditional probabilty of } level_{i} \text{ given label=0: } P_{0}=\frac{n_{0}}{N_{0}}$$

As a result, when woe is close to 0, the predictive power of that level (in this case, token) is low, because the proportions P0 and P1 are close. Therefore, this level is useless in the task of classifing documents. The higher the woe value in absolute value, the higher the predictive power:

When woe>0, P<sub>1</sub>/P<sub>0</sub>>0, P<sub>1</sub>>P<sub>0</sub>: This level allows to identify label=1 documents, and viceversa.

But what is high enough?, let's do some math:
$$ woe = \alpha; log(\frac{P_{1}}{P_{0}})= \alpha; \frac{P_{1}}{P_{0}}=e^{ \alpha}$$

$$\alpha=0.1 => \frac{P_{1}}{P_{0}}=1.1,  P_{1} \text{ and } P_{0} \text{ are almost the same}$$
$$\alpha=0.25 => \frac{P_{1}}{P_{0}}=1.28,  P_{1} \text{ is 1.28 times } P_{0}$$
$$\alpha=0.5 => \frac{P_{1}}{P_{0}}=1.65,  P_{1}\text{ is 1.65 times } P_{0}$$

Also take into account that the log function is not simetric! (as the following plot reminds us)


In [29]:
//x goes from 0 (P1=0 and P0=1) to infinity (P1=close to one and P0= close to 0) so log goes from -infinity to +infity
//log(0/x) is infinite
//log(x/0) is NaN, but in our computation x/0 will be mapped to -infinite
//both will be mapped to special categories
val log_plot = spark.range(1, 1000).withColumn("x", $"id"/lit(100)).withColumn("log", log($"x"))

In [30]:
%%brunel
data("log_plot") line x(x) y(log) sort(id)
title("Log plot") axes(x:'x',y:'log(x)') interaction(none)

Almost every metrics is computed, we only need to add some variables: 

In [31]:
val stats = corpus.map(record => (record._3, 1) ).reduce((a, b) => (a._1+b._1, a._2+b._2) )
val N1 = stats._1.toLong 
val N = stats._2
val N0 = N - N1

println("Total number of documents: " + N )
println("Total number of events: " + N1)
println("Total number of non-events: " + N0)

Total number of documents: 855
Total number of events: 139
Total number of non-events: 716


Moreover, take into account that there may be tokens that appear only in label=1 documents and viceversa, so, they will yield null values

In [32]:
//When P1=(n1/N1)=0, so e=0, woe is assigned to -infinity
rank_df.filter($"e" === lit(0)).show(5)

+----------+-----+---------+---+----+-----+------------------+---------+----------+------------+----+
|     token|ndocs|frequency|  e|  ne|eprop|           logFreq|freq_bins|ndocs_bins|logfreq_bins|rank|
+----------+-----+---------+---+----+-----+------------------+---------+----------+------------+----+
|      sara|   39|      127|0.0|39.0|  0.0| 2.103803720955957|        2|         1|           2| 266|
|      tana|   39|      101|0.0|39.0|  0.0|2.0043213737826426|        2|         1|           2| 362|
|       wpd|   11|      100|0.0|11.0|  0.0|               2.0|        2|         1|           2| 369|
|shackleton|   31|       97|0.0|31.0|  0.0|1.9867717342662448|        1|         1|           1| 373|
|    folder|    2|       93|0.0| 2.0|  0.0|1.9684829485539352|        1|         0|           1| 392|
+----------+-----+---------+---+----+-----+------------------+---------+----------+------------+----+
only showing top 5 rows



In [33]:
//When P0=(n0/N0)=0, as a result ne=0, woe is assigned to infinity
rank_df.filter($"ne" === lit(0)).show(5)

+-------+-----+---------+---+---+-----+------------------+---------+----------+------------+----+
|  token|ndocs|frequency|  e| ne|eprop|           logFreq|freq_bins|ndocs_bins|logfreq_bins|rank|
+-------+-----+---------+---+---+-----+------------------+---------+----------+------------+----+
|numerix|    1|       41|1.0|0.0|  1.0|1.6127838567197355|        1|         0|           1| 967|
| schenk|    1|       33|1.0|0.0|  1.0|1.5185139398778875|        1|         0|           1|1203|
|  solar|    2|       29|2.0|0.0|  1.0| 1.462397997898956|        1|         0|           1|1369|
|pergher|    6|       27|6.0|0.0|  1.0|1.4313637641589874|        1|         0|           1|1455|
| unfair|    4|       27|4.0|0.0|  1.0|1.4313637641589874|        1|         0|           1|1468|
+-------+-----+---------+---+---+-----+------------------+---------+----------+------------+----+
only showing top 5 rows



In [34]:
//import org.apache.spark.sql.types._
//val WOEThreshold = 0.5

val rawAnalytics_df = rank_df.withColumn("eprop", $"e"/$"frequency").
    withColumn("N1", lit(N1)).withColumn("N0", lit(N0)).
    withColumn("P1",$"e"/$"N1").withColumn("P0",$"ne"/$"N0").
    withColumn("x",when($"P1" ===lit(0.0), lit(Double.NegativeInfinity)).
                when($"P0" === lit(0.0), lit(Double.PositiveInfinity)).
                otherwise($"P1"/$"P0")).
    withColumn("WOEValue",when($"x"===lit(Double.NegativeInfinity), lit(Double.NegativeInfinity)).
               when($"x"===lit(Double.PositiveInfinity),lit(Double.PositiveInfinity)).
               otherwise(log($"P1"/$"P0"))).withColumn("token_length", length($"token").cast(DoubleType)).
    withColumn("WOEAbs", abs($"WOEValue")).
    withColumn("flg_singleLabel", when($"e" ===lit(0.0) || $"ne" === lit(0.0), 1).otherwise(0))
   
rank_df.unpersist()
rawAnalytics_df.persist()
println("WOE Computation: ")
rawAnalytics_df.select($"token", $"frequency", $"e", $"ne", $"eprop", $"N0", $"N1", $"P0", $"P1", $"WOEValue").show(5)

WOE Computation: 
[Stage 81:=============================>                            (1 + 1) / 2]+-----+---------+----+-----+--------------------+---+---+-------------------+-------------------+-------------------+
|token|frequency|   e|   ne|               eprop| N0| N1|                 P0|                 P1|           WOEValue|
+-----+---------+----+-----+--------------------+---+---+-------------------+-------------------+-------------------+
|enron|     4977|95.0|417.0| 0.01908780389793048|716|139| 0.5824022346368715| 0.6834532374100719|0.15999690363169333|
|  ect|     4131|52.0|309.0|0.012587751149842653|716|139|0.43156424581005587|0.37410071942446044|-0.1428913244863642|
|  com|     3625|80.0|252.0|0.022068965517241378|716|139|0.35195530726256985| 0.5755395683453237|0.49180378099241207|
|  hou|     2051|46.0|283.0| 0.02242808386153096|716|139| 0.3952513966480447|0.33093525179856115|-0.1775992673241888|
|    e|     1318|66.0|202.0| 0.05007587253414264|716|139|0.28212290502793297

In [35]:
// The code was removed by DSX for sharing.

In [36]:
import com.ibm.ibmos2spark.bluemix
val file = "enron_analytics_rawrank.csv"
//credentials_1 is a code snippet from  files tab
val configurationname1 = "os_" + (credentials_1("project").replace("object_storage_", "").replace("_", "") ) + "_configs"
//configurationname: 0s_XXXXXX_configs where XXXXX is crendentils("project") substrancting ("object_storage_" and removing  every "_")

var bmos = new bluemix(sc, configurationname1, credentials_1)
val path = bmos.url(credentials_1("container"), file)

In [37]:
println("Saving a Spark DataFrame as csv file to Bluemix Cloud Object Storage:")
println("Container name: " + credentials_1("container"))
println("File name: " + file)
rawAnalytics_df.drop("logFreq", "x").coalesce(1).write.mode("overwrite").
    option("delimiter",",").
    option("header","true").
    option("treatEmptyValuesAsNulls", "true").
    csv(path)

Saving a Spark DataFrame as csv file to Bluemix Cloud Object Storage:
Container name: DSLABTextAnalytics
File name: enron_analytics_rawrank.csv
